# Introduction to Machine Learning

*Machine Learning* is the foundation for most artifical intelligence solutions, and the creation of an intelligent solution often begins with the use of *supervised* machine learning to train a predictive model using historic data that you have collected.

Supervised machine learning techniques involve training a model to operate on a set of *features* and predict a *label* using a dataset that includes some already-known label values. The training process *fits* the features to the known labels to define a general function that can be applied to new features for which the labels are unknown, and predict them. You can think of this function like this, in which ***y*** represents the label we want to predict and ***x*** represents the features the model uses to predict it.

$$y = f(x)$$


That seems a little abstract, so let's take a look at an example.

Suppose *Adventure Works Cycles* is a business that rents cycles from a number of locations around the city. It might be useful to leverage historic data to train a model that predicts daily rental demand in order to help ensure that sufficient staff and cycles are available. The demand would be measured in terms of the number of rentals for a given day (this is our *y* value, or *label*), and the factors that affect demand might include things like the day of the week, the temperature, the level of rainfall, humidity level, windspeed, and so on (these are our *x* values, or *features*). So what we're looking for is a function (*f*) that perfoms some kind of calculation with the features (*x*) that produces the label (*y*) as a result.

This kind of machine learning, where the label you're trying to predict is a numeric value (in this case, the number of rentals), is called *regression*. Other kinds of machine learning include *classification* (in which the model predicts the probability that an entity belongs to a specific category, or *class*) and *time-series forecasting* (which is similar to regression, but takes into account trends over time).

The specific operation that the ***f*** function performs on *x* to calculate *y* depends on a number of factors, including the type of model you are trying to create, any pre-processing steps applied to the data, and the specific algorithm used to train the model. Data scientists expend a lot of effort exploring and pre-processing data, and trying various types of model-training algorithm to produce accurate models, which is time consuming, and often makes inefficient use of expensive compute hardware.

## Azure Machine Learning

Azure Machine Learning is a cloud-based platform for building and operating machine learning solutions in Azure. It includes a wide range of features and capabilities that help data scientists prepare data, train models, publish predictive services, and monitor their usage. Most importantly, it helps data scientists increase their efficiency by automating many of the time-consuming tasks associated with training models; and it enables them to leverage cloud compute resources that scale effectively to handle large volumes of data while incurring costs only when actually used.

## Creating an Azure Machine Learning Workspace

To use Azure Machine Learning, you create a *workspace* in your Azure subscription. You can then use this workspace to manage data, compute resources, code, models, and other artefacts related to your machine learning workloads.

Follow these steps to create a workspace:

1. Sign into the [Azure portal](https://portal.azure.com) using your Microsoft credentials.
2. Click the **&#65291;Create a resource** button, search for *Machine Learning*, and create a new **Machine Learning** resource the following settings:
    - **Workspace Name**: *A unique name of your choice*
    - **Subscription**: *Your Azure subscription*
    - **Resource group**: *Create a new resource group with a unique name*
    - **Location**: *Choose any available location*
    - **Workspace edition**: Enterprise
3. Wait for your workspace to be created. Then go to it in the portal.

## Managing Azure Machine Learning Resources

Now that you have an Azure Machine Learning workspace, you can use it to manage the various assets and resources you need to create machine learning solutions. At its core, Azure Machine Learning is a platform for training and managing machine learning models, for which you need two fundamental things: data from which to train the models, and compute on which to run the training process. You'll manage both of these for the cycle rental prediction model using the Azure Machine Learning *studio* web interface.

1. In the [Azure portal](https://portal.azure.com), in the **Overview** page for your Azure Machine Learning workspace, click the link to launch Azure Machine Learning studio. Alteratively, browse to [https://ml.azure.com](https://ml.azure.com]), sign in using your Microsoft credentials, and select your Azure subscription and Azure Machine Learning workspace.
2. View the **Compute** page (under **manage**). This is where you manage the compute targets for your data science activities. There are four kinds of compute resource you can create:
    - **Compute Instances** (sometimes called *Notebook VMs*): Development workstations for working with data and models.
    - **Training Clusters**: Scalable clusters of virtual machines for on-demand processing of model training code.
    - **Inference Clusters**: Deployment targets for predictive services that use your trained models.
    - **Attached Compute**: Links to existing Azure compute resources, such as Virtual Machines or Azure Databricks clusters.
3. Switch to the **Training Clusters** tab, and add a new training cluster with the following settings:
    - **Compute name**: aml-cluster
    - **Virtual Machine size**: Standard_DS12_v2
    - **Virtual Machine priority**: Dedicated
    - **Minimum number of nodes**: 0
    - **Maximum number of nodes**: 4
    - **Idle seconds before scale down**: 120
4. While the training cluster is being created, view the **Datasets** page (under **Assets**), and create a new dataset from web files with the following settings:
    - **Web URL**: https://aka.ms/bike-rentals
    - **Name**: bike-rentals
    - **Dataset type**: Tabular
    - **Description**: Bike rental data
    - **Settings and preview**: Review the automatically detected settings.  Select **Use headers from the first file** under Column Headers.
    - **Schema**: Review the default column selections and data types.
4. After the dataset has been created, open it and view the **Explore** page to see a sample of the data. This data contains historical features and labels for bike rentals.

> **Citation**: *This data is derived from [Capital Bikeshare](https://www.capitalbikeshare.com/system-data) and is used in accordance with the published data [license agreement](https://www.capitalbikeshare.com/data-license-agreement).*

## Training a Model Using Automated Machine Learning

Azure Machine Learning includes an *automated machine learning* capability that leverages the scalability of cloud compute to automatically try multiple model-training algorithms and pre-processing techniques in parallel to find the best performing model for your data. Perform the following steps to use it to train a model that predicts bike rentals.

1. In [Azure Machine Learning studio](https://ml.azure.com), view the **Automated ML** page (under **Author**).
2. Create a new automated ML run with the following settings:
    - **Dataset**: bike-rentals
    - ** Experiment name**: auto-train-bike-rental
    - **Target column**: rentals
    - **Training compute target**: aml-cluster
    - **Task type**: Regression
    - **Additional configuration settings:**
        - **Primary metric**: Normalized root mean square error (more about this later!)
        - **Automated featurization**: Selected
        - **Explain best model**: Unselected
        - **Blocked algorithms**: *block <u>all</u> other than **RandomForest** and **LightGBM** - normally you'd want to try as many as possible, but doing so will take more time than we have available in this lab!*
3. When you finish submitting the automated ML run details, it will start automatically. Wait for the run status to change from *Preparing* to *Running* (this may take 10 minutes or so, as the cluster nodes need to be initialized before training can begin - now might be a good time for a coffee break!). You may need to click **&#8635;Refresh** periodically.
4. When the run status changes to *Running*, click the **Models** tab and observe as each possible combination of training algorithm and pre-processing steps is tried and the performance of the resuting model is evaluated. The page will auto-refresh periodically, but you can also click **&#8635;Refresh**.
5. After a few models have been trained and evaluated, click **&#10005; Cancel**. and cancel the remaining iterations.

## Reviewing and Deploying the Best Model

Even though you cancelled the automated machine learning run, some models were trained; so you can review the best performing one and deploy it as a predictive service.

1. On the **Details** tab of the automated machine learning run, note the recommended model.

    This recommendation is based on the performance metric you specified (Normalized root mean square error). To calculate this, the training process used some of the data to train the model, and applied a technique called *cross-validation* to test the trained model with data it wasn't trained with (but for which the actual label value is known), and compare the predicted value with the actual known value. The difference between the predicted and actual value (known as the *residuals*) indicates the amount of *error* in the model, and our performance metric is calculated by squaring the errors across all of the test cases, finding the mean of these squares, and then taking the square root. What all of this means is that smaller this value is, the more accurately the model is predicting.

2.  Click **View model details**, and note that you can see all of the run metrics that give statistical information about the performance of the model.
3. Click the **Visualizations** tab and review the charts that show the performance of the model by comparing the predicted values against the true values, and showing the *residuals* (differences between predicted and actual values) as a histogram.
4. Return to the **Model details** tab, and click **Deploy model**. Then deploy the model with the following settings:
    - **Name**: predict-rentals
    - **Description**: Predict cycle rentals
    - **Compute type**: ACI
    - **Enable authentication**: Selected.
5. Wait for the deployment to complete - this may take a few minutes.
6. In Azure Machine Learning studio, view the **Endpoints** page and find the **predict-rentals** real-time endpoint.
7. Click the **predict-rentals** endpoint and verify that the Deployment state is *Healthy*. Then click the **Consume** tab and note the information there. You need this to connect to your deployed service from a client application.
8. Copy the REST endpoint for your service (you can use the &#10697; link next to it), and paste it in the code below (replacing YOUR_ENDPOINT).
9. Copy the Primary Key for your service and paste it in the code below, replacing YOUR_KEY.
10. Run the code in the cell below by clicking its green <span style="color:green">&#9655</span> button.

In [ ]:
endpoint = 'YOUR_ENDPOINT' # Replace with your endpoint
key = 'YOUR_KEY' # Replace with your key

print ('Ready to use', endpoint, 'with key', key)

Now you're ready to use your service.

Run the code cell below, which defines features for a five day period using hypothetical weather forecast data, and uses the **predict-rentals** service you created to predict cycle rentals for those five days

In [ ]:
import json
import requests

# Features based on five-day weather forecast
x_new = [[1,1,2022,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446],
         [2,1,2022,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539],
         [3,1,2022,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309],
         [4,1,2022,1,0,2,1,1,0.2,0.212122,0.590435,0.160296],
         [5,1,2022,1,0,3,1,1,0.226957,0.22927,0.436957,0.1869]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = {"Content-Type":"application/json",
          "Authorization":"Bearer " + key}

# Get the predictions
response = requests.post(endpoint, input_json, headers=headers)

# If we got a valid response, print it
if response.status_code == 200:
    predicted_rentals = json.loads(response.json())
    print("Predictions:")
    for i in range(len(x_new)):
        print (" Day: {}. Predicted rentals: {}".format(i+1, round(predicted_rentals["result"][i])))
else:
    print(response)

Your machine learning model is predicting rentals based on the features you submit to your service, making it possible for Adventure Works Cycles to ensure they have enough staff and cycles in place to meet demand.

> **Learn More**: We've only scratched the surface of Azure Machine Learning in this lab. To learn more, see the [Azure Machine Learning documentation](https://docs.microsoft.com/azure/machine-learning/)